In [1]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [2]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v39_100M.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 4000.0,
    "DACInterpolationRate": 40,
    "DACNCO": 0.0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 20,
    "ADCNCO": -0.0,
}
ZCU216_CONFIG.update(NEW_CONFIG)

adc_datapath/t226/data_mover_ctrl
adc_datapath/t226/fifo_count
axi_gpio_led
dac_datapath/t230/data_mover_ctrl
dac_datapath/t230/fifo_count
usp_rf_data_converter
zynq_ultra_ps_e


In [3]:
import numpy as np

qam_order = "16QAM"  # Have to change it manually

for nco in np.arange(50.0, 2000.0, 100.0):

    # Update config
    inv_sinc = 1
    atten = 0.0
    ZCU216_CONFIG['DACNCO'] = nco
    ZCU216_CONFIG['InvSincFIR'] = inv_sinc
    ZCU216_CONFIG['OFDM_ATTEN_DB'] = atten

    ZCU216_CONFIG['CONFIG_NAME'] = "Tx_100M_BW" + \
        "_" + qam_order + "_" + str(nco) + "_" + \
        str(atten) + "_" + str(inv_sinc)

    rfdc_t = RfdcTask(ol)

    for task in [rfdc_t]:
        task.start()
        task.join()

    led_t = BlinkLedTask(ol)
    tx_t = SingleChTxTask(ol, tx_file="./wave_files/Tx.npy")

    parallel_task = [led_t, tx_t]
    for task in parallel_task:
        task.start()

    print(
        f"qam_order {qam_order}, nco {nco}, atten {atten}, inv_sinc {inv_sinc}")
    dummy = input('Type anything and press Enter to continue.')

    for task in parallel_task:
        task.stop()

2024-11-16 23:26:40,712 - root - INFO - DAC tile 0 DAC block 0 is NOT enabled!
2024-11-16 23:26:40,717 - root - INFO - DAC tile 0 DAC block 1 is NOT enabled!
2024-11-16 23:26:40,720 - root - INFO - DAC tile 0 DAC block 2 is NOT enabled!
2024-11-16 23:26:40,723 - root - INFO - DAC tile 0 DAC block 3 is NOT enabled!
2024-11-16 23:26:40,832 - root - INFO - DAC tile 1 DAC block 0 is enabled!
2024-11-16 23:26:40,835 - root - INFO - DAC tile 1 DAC block 1 is NOT enabled!
2024-11-16 23:26:40,837 - root - INFO - DAC tile 1 DAC block 2 is NOT enabled!
2024-11-16 23:26:40,840 - root - INFO - DAC tile 1 DAC block 3 is NOT enabled!
2024-11-16 23:26:40,887 - root - INFO - DAC tile 2 DAC block 0 is enabled!
2024-11-16 23:26:40,889 - root - INFO - DAC tile 2 DAC block 1 is NOT enabled!
2024-11-16 23:26:40,892 - root - INFO - DAC tile 2 DAC block 2 is NOT enabled!
2024-11-16 23:26:40,894 - root - INFO - DAC tile 2 DAC block 3 is NOT enabled!
2024-11-16 23:26:40,897 - root - INFO - DAC tile 3 DAC block

FileNotFoundError: The specified file does not exist: ./tmp_files/Tx.npy